## From Data Lake to Data Warehouse

### What the Notebook Does

- **Data Loading and Parsing:**  
  The notebook reads HTML files containing newspaper articles along with their corresponding metadata from a CSV file. It then uses BeautifulSoup to parse the HTML and extract only the relevant content needed for further analysis.

- **Data Processing and Preparation:**  
  The extracted content is processed to isolate the contexts in which the term "klima" appears. This includes capturing the surrounding text to better understand the usage and meaning of the word in each article.

- **Data Storage:**  
  The processed data is structured and stored in a SQLite database with two tables. This ensures that the data is organized, easily accessible, and ready for further analysis. It will also export the data as csv for an easy import to other programms.

### Data Format

- **Table: newspaper**  
  Stores metadata about each newspaper's main page, including the publication details corresponding to a single day.  
  Each entry represents the main page of a newspaper for one day, as the dataset is derived from crawling the main page rather than individual articles. 
  **Columns:**  
  - `newspaper_id`  
  - `newspaper_name`  
  - `data_published`  
  - `klima_mentions_count`

- **Table: context**  
  Contains detailed text snippets surrounding the target word "klima". The id refers to a newspaper (main page) from one specific day. 
  **Columns:**  
  - `newspaper_id`  
  - `pre_context`  
  - `post_context`  
  - `prefix`  
  - `suffix`

### Why This Approach

- **Focused Analysis:**  
  By isolating the contexts where "klima" is mentioned, the notebook prepares data specifically tailored to analyze the evolution of the term's usage over time.

- **Data Organization:**  
  Storing data in a structured SQLite database facilitates efficient querying and analysis, ensuring that subsequent analytical processes can be performed seamlessly.

- **Reproducibility and Scalability:**  
  This clear separation of tasks—from data extraction to storage—supports a reproducible workflow that can easily be extended or modified for future analytical targets.

For additional details and background, please refer to the README file.


In [ ]:
import os
import sys
import glob
import csv

# Add custom library path relative to notebook location
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
sys.path.append(os.path.join(notebook_dir, "..", "pylib"))

import pandas as pd
from handle_sqlite import read_table_as_dataframe
from handle_data_processing import batch_process_newspapers

db_path = os.path.join(notebook_dir, "..", "data_output", "dwh_data.db")

### Load All the Newspapers

In this section, we load the CSV files that contain details for each newspaper, such as file path, date, and HTTP status code. Each file represents data from one day.

In [2]:
# Use glob to list all CSV files in the specified directory that follow a date format in their names

csv_dir = os.path.join(notebook_dir, "..", "data_input", "data-lake")
csv_files = glob.glob(os.path.join(csv_dir, "*-*.csv"))

We sort the files based on the date portion of the filename (ignoring the directory path). This help in the long taking processing step to easily track progress from the start date to the end date.

In [3]:
# we sort by the filename which contains the date, ignoring the directory path to make the sort efficient
csv_files.sort(key=lambda f: f.split('/')[-1])

# Output the total count of days (CSV files) to confirm the number of entries
print(f'Count of total days: {len(csv_files)}')

Count of total days: 8


Now, we will read the CSV files one by one to retrieve the HTML file paths, including only those with a status code of 200 (OK).

In [4]:
# Initialize an empty DataFrame to store newspaper data
newspapers = []

# Process each CSV file
for csv_file in csv_files:
    # Open and read the CSV file
    with open(csv_file, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)

        # For each row, check if the status code is 200 (OK) and append the newspaper data to the list
        for row in reader:
            if row['status'] == '200':
                newspapers.append({
                    'name': row['name'],
                    'date': row['date'],
                    'file_name': row['file_name'],
                    'encoding': row['encoding']
                })


For verification: Display the first two newspaper entries to inspect the structure

In [5]:
# Print the first two entries to verify the structure of the newspaper data
newspapers[:2]

[{'name': 'sz',
  'date': '2025-02-01',
  'file_name': 'data-lake/2025-02-01-sz.html',
  'encoding': 'utf-8'},
 {'name': 'zeit',
  'date': '2025-02-01',
  'file_name': 'data-lake/2025-02-01-zeit.html',
  'encoding': 'UTF-8'}]

### Do the Actual Processing with Batch Processing and Multiprocessing

Here we process the newspaper data in batches using multiprocessing. This step prepares the data by extracting the relevant HTML content and storing the results in a SQLite database.

In [6]:
batch_process_newspapers(
    newspapers,
    batch_size=512,
    num_workers=12,
    db_path=os.path.join(notebook_dir, "..", "data_output", "dwh_data.db"),
    input_path_prefix=os.path.join(notebook_dir, "..", "data_input")
)

2025-09-12 22:32:37,887 - INFO - Processing at date 2025-02-08.
2025-09-12 22:32:37,897 - INFO - Data saved to table 'newspapers' in '/Users/edwinw/Documents/DBU Uni/wortwandel/notebooks/../data_output/dwh_data.db' successfully.
2025-09-12 22:32:37,901 - INFO - Data saved to table 'context' in '/Users/edwinw/Documents/DBU Uni/wortwandel/notebooks/../data_output/dwh_data.db' successfully.


### Check the Saved Data

After processing, we load the saved data from the SQLite database to verify that the data has been stored correctly.

In [7]:

# Read the 'newspapers' table from the database and display the first few rows
meta_data = read_table_as_dataframe("newspapers", db_path)
meta_data.head()

2025-09-12 22:32:38,234 - INFO - Data read from table 'newspapers' in '/Users/edwinw/Documents/DBU Uni/wortwandel/notebooks/../data_output_stick/dwh_data.db' successfully.


,newspaper_name,data_published,klima_mentions_count,newspaper_id
0,sz,2021-04-01,6,1
1,zeit,2021-04-01,4,2
2,faz,2021-04-01,11,3
3,heise,2021-04-01,4,4
4,golem,2021-04-01,0,5


In [8]:
# Read the 'context' table from the database and display the first few rows
context_data = read_table_as_dataframe("context", db_path)
context_data.head()

2025-09-12 22:32:38,476 - INFO - Data read from table 'context' in '/Users/edwinw/Documents/DBU Uni/wortwandel/notebooks/../data_output_stick/dwh_data.db' successfully.


,pre_context,post_context,prefix,suffix,newspaper_id
0,im Zuge des,werden dürfte. Kommentar,,wandels,1
1,Digital Alles zur,Das Wichtigste auf,,krise,1
2,einer Seite Der,Streitgespräch Wirtschaftswachstum und,SZ,monitor,1
3,Streitgespräch Wirtschaftswachstum und,04:00 Video Warum,,krise,1
4,Golfstrom für das,so wichtig ist,,,1


### Export the Processed Data as CSV Files (optional)

Finally, we export the stored data as CSV files to facilitate further analysis in other programs that can't import sqlite files.

In [9]:
import datetime

# Get today's date in YYYY-MM-DD format
today = datetime.datetime.now().strftime("%Y-%m-%d")

# Export the metadata and context data as CSV files, embedding today's date in the filenames
meta_data.to_csv("dwh_meta_{today}.csv", index=False)
context_data.to_csv("dwh_context_{today}.csv", index=False)

In [10]:
# Check the number of unique newspaper names in the metadata (64)
# This helps verify that each newspaper is uniquely represented
meta_data.newspaper_name.nunique()

64

In [11]:
# Check the number of unique publication dates in the metadata (must equal 'Count of total days': 1401)
# This ensures that we have distinct entries for each day the main page was crawled
meta_data.data_published.nunique()

1401